In [9]:
import numpy as np
from scipy import stats
import tensorflow as tf

In [5]:
with open('ytest.csv', 'r') as f:
    y_test = np.genfromtxt(f, delimiter=',')
    
with open('ypred_Lin.csv', 'r') as f:
    y_pred_Lin = np.genfromtxt(f, delimiter=',')

with open('ypred_KNN.csv', 'r') as f:
    y_pred_Knn = np.genfromtxt(f, delimiter=',')

with open('ypred_DNN.csv', 'r') as f:
    y_pred_Dnn = np.genfromtxt(f, delimiter=',')

with open('ypred_CNN.csv', 'r') as f:
    y_pred_Cnn = np.genfromtxt(f, delimiter=',')


In [35]:
def modeCombine(y_pred_Lin, y_pred_Knn, y_pred_Dnn, y_pred_Cnn):
    y_pred = []
    for i in range(len(y_pred_Lin)):
        temp = [y_pred_Lin[i], y_pred_Knn[i], y_pred_Dnn[i], y_pred_Cnn[i]]
        if stats.mode(temp)[0] > 1:
            y_pred.append(stats.mode(temp)[0][0])
        else:
            y_pred.append(y_pred_Cnn[i])
    return y_pred

In [36]:
from statistics import mode
from sklearn.metrics import confusion_matrix

y_pred = modeCombine(y_pred_Lin, y_pred_Knn, y_pred_Dnn, y_pred_Cnn)

cm = confusion_matrix(y_test, y_pred)

correct = 0
for i in range(len(cm)):
    correct += cm[i][i]

r = round((correct) / np.sum(cm), 4)
print(f'Classifiation Rate: {r}')

np.savetxt('data_Combined.csv', cm, delimiter=',')

Classifiation Rate: 0.8807


In [42]:
def probCombine(y_pred_Lin, y_pred_Knn, y_pred_Dnn, y_pred_Cnn):
    y_pred_Lin = np.asarray(y_pred_Lin).astype('float32')
    y_pred_Knn = np.asarray(y_pred_Knn).astype('float32')
    y_pred_Dnn = np.asarray(y_pred_Dnn).astype('float32')
    y_pred_Cnn = np.asarray(y_pred_Cnn).astype('float32')

    labels_Lin = tf.reshape(tf.one_hot(y_pred_Lin, 62), [-1, 62])
    labels_Knn = tf.reshape(tf.one_hot(y_pred_Knn, 62), [-1, 62])
    labels_Dnn = tf.reshape(tf.one_hot(y_pred_Dnn, 62), [-1, 62])
    labels_Cnn = tf.reshape(tf.one_hot(y_pred_Cnn, 62), [-1, 62])

    prob_Lin = 0.0
    prob_Knn = 0.5
    prob_Dnn = 0.5
    prob_Cnn = 1.0

    y_pred = []
    for i in range(len(y_pred_Lin)):
        temp = prob_Lin * labels_Lin[i] + prob_Knn * labels_Knn[i] + prob_Dnn * labels_Dnn[i] + prob_Cnn * labels_Cnn[i]
        y_pred.append(np.argmax(temp, axis = -1))

    return y_pred

In [44]:
from statistics import mode
from sklearn.metrics import confusion_matrix

y_pred = probCombine(y_pred_Lin, y_pred_Knn, y_pred_Dnn, y_pred_Cnn)

cm = confusion_matrix(y_test, y_pred)

correct = 0
for i in range(len(cm)):
    correct += cm[i][i]

r = round((correct) / np.sum(cm), 4)
print(f'Classifiation Rate: {r}')

np.savetxt('data_Combined.csv', cm, delimiter=',')

Classifiation Rate: 0.9363
